In [2]:
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiler_version())
print(get_compiling_cuda_version())
import mmdet
from mmdet.apis import set_random_seed
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector

import random
import numpy as np
from pathlib import Path
import copy
import json
from pycocotools.coco import COCO
import os

/home/chenyang/software/miniconda3/envs/pytorch/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1.11.0 True
GCC 7.3
11.3


In [3]:
seed = 42

"""Sets the random seeds."""
set_random_seed(seed, deterministic=False)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
os.environ['PYTHONHASHSEED'] = str(seed)

In [4]:
type2id = dict()

for i in range(-100, 101):
    type2id[str(i)] = i + 100
for up in [chr(i) for i in range(65, 91)]:
    type2id[up] = ord(up) - 65 + 201
for low in [chr(i) for i in range(97, 123)]:
    type2id[low] = ord(low) - 97 + 227

In [5]:
from mmcv import Config

cfg_path = "../../diagram_detection/code/job8_cascade_rcnn_r101_fpn_1x/job8_cascade_rcnn_r101_fpn_1x_coco.py"

cfg = Config.fromfile(cfg_path)

In [6]:
import numpy as np
from tqdm import tqdm
import json  # for dumping json serialized results
import zipfile  # for creating submission zip file
import pandas as pd
import cv2
import os
from matplotlib import pyplot as plt
from mmdet.apis import inference_detector, init_detector, show_result_pyplot

In [7]:
checkpoint = "../../diagram_detection/code/job8_cascade_rcnn_r101_fpn_1x/best_bbox_mAP_epoch_19.pth"

model_test = init_detector(cfg, checkpoint, device = "cuda:0")

load checkpoint from local path: ../../diagram_detection/code/job8_cascade_rcnn_r101_fpn_1x/best_bbox_mAP_epoch_19.pth


/home/chenyang/code/CCKS2022/diagram_detection/code/mmdetection/mmdet/apis/inference.py:50: UserWarning: Class names are not saved in the checkpoint's meta data, use COCO classes by default.
  warnings.warn('Class names are not saved in the checkpoint\'s '


In [8]:
class_name = list(type2id.keys())

In [9]:
def get_xyxy_from_pic(img_full_name, model, score_thresh):
    xy_list = []
    categorys = []
    result = inference_detector(model, img_full_name)
    for i in range(len(class_name)):
        if len(result[i]) != 0:
            for j in result[i]:
                j = np.array(j).tolist()
                if j[-1] >= score_thresh:
                    categorys.append(class_name[i])
                    x_min, y_min, x_max, y_max = j[0], j[1], j[2], j[3]
                    x_mid = (x_max + x_min) / 2
                    y_mid = (y_min + y_max) / 2
                    width = x_max - x_min
                    height = y_max - y_min
                    xy_list.append([int(x_mid), int(y_mid), int(width), int(height)])
    return categorys, xy_list

In [10]:
json_file = dict()

max_len = 0

for folder in ['train', 'val', 'test']:
    for img_name in os.listdir(os.path.join("../data/ccksdata/", folder)):
        img_full_name = os.path.join("../data/ccksdata/", folder, img_name)
        cat, xy_list = get_xyxy_from_pic(img_full_name, model_test, score_thresh=0.7)
        json_file[img_name] = list()
        if len(cat) > max_len:
            max_len = len(cat)
        # for i in range(len(cat)):
        #     tmp = dict()
        #     tmp["shape_attributes"] = dict()
        #     tmp["shape_attributes"]["x"] = xy_list[i][0]
        #     tmp["shape_attributes"]["y"] = xy_list[i][1]
        #     tmp["shape_attributes"]["width"] = xy_list[i][2]
        #     tmp["shape_attributes"]["height"] = xy_list[i][3]
        #     tmp["region_attributes"] = dict()
        #     tmp["region_attributes"]["Obj_id"] = str(i)
        #     tmp["region_attributes"]["value"] = cat[i]
        #     json_file[img_name].append(tmp)

/home/chenyang/code/CCKS2022/diagram_detection/code/mmdetection/mmdet/datasets/utils.py:70: UserWarning: "ImageToTensor" pipeline is replaced by "DefaultFormatBundle" for batch inference. It is recommended to manually replace it in the test data pipeline in your config file.
  'data pipeline in your config file.', UserWarning)
libpng warning: zTXt: incorrect header check
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: profile 'icc': 0h: PCS illuminant is not D50


In [11]:
max_len

44

: 

In [12]:
json.dump(json_file, open("./diagram_detection.json", "w"))

/home/chenyang/software/miniconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:1: ResourceWarning: unclosed file <_io.TextIOWrapper name='./diagram_detection.json' mode='w' encoding='UTF-8'>
  """Entry point for launching an IPython kernel.
